In [1]:
#Import libraries
from bqplot import *
from ipywidgets import *
import pandas as pd

# Frequency planning tool for RFSoC devices

This Jupyter notebook contains the following:
   
    1) RF Analogue to Digital frequency plan chart
    2) RF Digital to Analogue frequency plan chart
    3) Digital Down Conversion Spectrum
    4) Digital Up Conversion Spectrum

These tools will allow the user to create an in depth frequency plan for any RF project

# User guide

In all charts sliders are used to select the Sampling frequency, Input frequency and the Bandwidth of the input signal. 
In the DUC and DDC charts sliders are also available to set the Frequency of the NCO and the level of noise (in dB) in the Harmonic spurs HD2 and HD3. 
The DDC chart features radio buttons that allow the user to select the number of interleaved ADC's available and the Decimation factor.

# 1) RFADC Frequency chart

The following chart provides the relevant information required for the RFADC process. 

In [2]:
#This function is used to calculate the frequency of the input signal in the 1st Nyquist zone
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth):
    
    #Calculate low and high bound of the signal
    Fin_low = Signal_center - (Signal_Bandwidth/2)
    Fin_High = Signal_center + (Signal_Bandwidth/2)
    
    #Calculate Nyquist rate
    Nyquist = float(Fs/2)
    
    #Set variable K depending on location of input signal 
    #K used in alias equation for folding signal back to 1st Nyquist zone
    if Fin_High <= Nyquist:
        K = 0
        
    elif Fin_High <= Fs + Nyquist:
        K = 1
    
    elif Fin_High <= 2*Fs + Nyquist:
        K = 2
        
    elif Fin_High <= 3*Fs + Nyquist:
        K = 3
        
    else:
        K = 4        
        
    #Calculate input alias
    Low_Input = (Fin_low - Fs*K)
    High_Input = (Fin_High - Fs*K)
    
    #Mirror input if value is negative
    if Low_Input < 0:
        High_Input_return=Low_Input*-1
        Low_Input_return=High_Input*-1
    
    #If positive set value to value as is from input alias calculations previous
    else:
        Low_Input_return = Low_Input
        High_Input_return = High_Input
 
    #If input signal reaches boundaries of 1st Nyquist zone, set equal to relative boundary
    if High_Input_return > Nyquist:
        High_Input_return = Nyquist
        
    if Low_Input_return < 0:
        Low_Input_return = 0
        
        
    #Return the input values
    return [Low_Input_return, High_Input_return]

In [3]:
#This function is used to calculate the harmonic spurs relative to the input signal
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Spur_Calculator(Fs, Signal_center, Signal_Bandwidth):

    #Call input signal function
    Input_info = Input_signal_calculator(Fs,Signal_center,Signal_Bandwidth)
    
    #Calculate Nyquist rate
    Nyquist = Fs/2
    
    #Set frequency bounds to returned values from function
    fin_low = Input_info[0]
    fin_high = Input_info[1]
    
    #Craete list for storing spur values
    HDlist=[]
    
    #Iterate through for loop to calculate spurs HD2 uptop HD5
    for n in range (2,6,1):
        
        #Calculate K vairable for folding spur signal back to first nyquist zone
        if fin_high*n <= Nyquist:
            K = 0
        
        elif fin_high*n <= Fs + Nyquist:
            K = 1
    
        elif fin_high*n <= 2*Fs + Nyquist:
            K = 2
        
        elif fin_high*n <= 3*Fs + Nyquist:
            K = 3
        
        elif fin_high*n <= 4*Fs + Nyquist:
            K = 4
            
        elif fin_high*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6

        
        #Calculate spur locations
        HDn_high = (fin_high *n) - (Fs*K)
        HDn_low = (fin_low *n) - (Fs*K)
        
        #IF HDn is negative, mirror result 
        if HDn_high <= 0:        

                    Mirror_HDn_low = HDn_low * -1;
                    Mirror_HDn_high = HDn_high * -1;
        
                    temp_list = (Mirror_HDn_low,Mirror_HDn_high) 
                    #Update list with calculated values 
                    HDlist.append(temp_list) 

        
        else:
                    temp_list = (HDn_high,HDn_low)
                    #Update list with calculated values
                    HDlist.append(temp_list)        
    
    #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_High = [HDlist[0][0],HDlist[1][0],HDlist[2][0],HDlist[3][0]]
    
    HD_High_list=[]
    
    for i in HD_High:
        if i > Nyquist:
            i = Nyquist
            temp_list = (i)
                    #Update list with calculated values
            HD_High_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
                    #Update list with calculated values
            HD_High_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_low = [HDlist[0][1],HDlist[1][1],HDlist[2][1],HDlist[3][1]]
    
    HD_low_list=[]
    
    for i in HD_low:
        if i < 0:
            i = 0
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)

    #Create spur values with relative list values
    HD2_value = [HD_High_list[0], HD_low_list[0]]
    HD3_value = [HD_High_list[1], HD_low_list[1]]
    HD4_value = [HD_High_list[2], HD_low_list[2]]
    HD5_value = [HD_High_list[3], HD_low_list[3]] 
    
    #This section of code checks to see if the spur locations overlap the input signal
    #If true then the overlap will be flagged in the output
    
    if (fin_low <= HD2_value[0])  and  (fin_high >= HD2_value[1]):
        print("HD2 overlaps")
    if (fin_low <= HD3_value[0])  and  (fin_high >= HD3_value[1]):
        print("HD3 overlaps")
    if (fin_low <= HD4_value[0])  and  (fin_high >= HD4_value[1]):
        print("HD4 overlaps")
    if (fin_low <= HD5_value[0])  and  (fin_high >= HD5_value[1]):
        print("HD5 overlaps")
    
    #return spur values
    return [HD2_value[0],HD2_value[1],HD3_value[0],HD3_value[1],HD4_value[0],HD4_value[1],HD5_value[0],HD5_value[1]]



In [4]:
#This function plots the RFADC chart
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def Plot_RFADC(Fs,Signal_center,Signal_Bandwidth):
    
    #Call function to calculate input signal
    Input_plot = Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth)
    
    #Call function to calculate the harmonic spurs
    Spur_plot = Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)
    
    #Calculate Nyquist rate
    Nyquist = Fs/2
    
    #Set y-coordinates for plot
    n1 = [5]
    n2 = [2,2]
    n3 = [3,3]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Set x and y data used for plotting
    spur_data = [Spur_plot[0],Spur_plot[1],Spur_plot[2],Spur_plot[3],Spur_plot[4],Spur_plot[5],Spur_plot[6],Spur_plot[7]]
    input_data = [Input_plot[0],Input_plot[1]]
   
    #Create array of y data to pass through plot tool
    y_data = [n2, n3, n4, n5]

    HD2_value = [Spur_plot[0], Spur_plot[1]]
    HD3_value = [Spur_plot[2], Spur_plot[3]]
    HD4_value = [Spur_plot[4], Spur_plot[5]]
    HD5_value = [Spur_plot[6], Spur_plot[7]] 
    
    spur_plot2 = [HD2_value,HD3_value,HD4_value,HD5_value]
    HD_index = ['HD2','HD3','HD4','HD5']
    

        
    #Set scale to linear
    x_sc = LinearScale()
    y_sc = LinearScale()
    
    #Label axis
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic numbers', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting data
    spurs = Lines(x=spur_plot2,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['green','cyan','purple','yellow','pink'],enable_move=False)  
    Fin_horz_plot = Lines(x=input_data,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[input_data[0],input_data[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[input_data[1],input_data[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_horz_plot_alias = Lines(x=input_data,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=[Nyquist,Nyquist],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    
    #Create tooltips for displaying data on hover
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'harmonic number'])
    tt_fim = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency image'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    
    #Create data tip shapes
    spurs.marker='square'
    Fin_horz_plot.marker='square'
    Nyquist_plot.marker='square'
    
    #Plot figure
    fig = Figure(title='RFADC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot,Fin_vert1_plot, Fin_vert2_plot,spurs,Nyquist_plot])
    spurs.tooltip=tt
    Fin_horz_plot.tooltip=tt_fim
    Nyquist_plot.tooltip=Nyquisttt
    tools=Toolbar(figure=fig)
    
    #Display figure and relevant tools
    display(fig,tools)
    
        
    frame1={'':['Fin','HD2','HD3','HD4','HD5'],
                'High':[input_data[1],spur_data[0],spur_data[2],spur_data[4],spur_data[6]],
                'Low':[input_data[0],spur_data[1],spur_data[3],spur_data[5],spur_data[7]]}
       
    
    tempFrame = pd.DataFrame(data=frame1)
    tempFrame.set_index('',inplace = True)    
    tempFrame.to_csv('ADC.csv', sep=',',index=False)

    
        #tempFrame.to_csv('ADC.csv', sep=',',index=False)
    #print(tempFrame)
        #'C:/Users/Christian/Desktop/CSV/
        
    import ipywidgets as widgets
    from IPython.display import clear_output
    def clicked(b):
    
    
        with out:
            clear_output(wait=True)
            display(tempFrame)
            
    out = widgets.Output()
    display(out)

    run_button = widgets.Button(
        description = 'Display numbers'
        )
    run_button.on_click(clicked)
    display(run_button)    


In [5]:
#Create sliders for selecting input information
import ipywidgets as widgets

Fs_slider = widgets.FloatSlider(value=3932.16,min=0,max=4000,step=10,description='Fs:')
Signal_Center_slider= widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal Center:')
Signal_Bandwidth = widgets.FloatSlider(value=100,min=50,max=200,step=50,description='Bandwidth:')

Fs_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Signal center:',continuous_update = False)
Signal_bandwidth_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Bandwidth:',continuous_update = False)
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((Signal_Bandwidth, 'value'), (Signal_bandwidth_text, 'value'))

left_box = VBox([Fs_slider, Signal_Center_slider, Signal_Bandwidth])
right_box = VBox([Fs_text, Signal_Center_text, Signal_bandwidth_text])
sel=HBox([left_box, right_box])

display(sel)

Fs= Fs_slider
Signal_center = Signal_Center_slider


#widgets.interact(Plot_RFADC,Fs=Fs_slider,Signal_center=Signal_Center_slider,Signal_Bandwidth=Signal_Bandwidth)

out = widgets.interactive_output(Plot_RFADC, {'Fs': Fs, 'Signal_center': Signal_center, 'Signal_Bandwidth': Signal_Bandwidth})
display(out)

Output()

# 2) RFDAC Frequency chart

The following chart provides the relevant information required for the RFDAC process.

In [6]:
#This function calculates the input signal and its image, to be displayed in the first two Nyquist zones
#Requires 3 input arguments, set from sliders
#Fs = Sampling Frequency, Signal_center = location of center of signal, Signal_bandwidth = Bandwidth of signal

def DAC_Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth):
    
    #Calculate low and high boundaries of signal
    Fin_low = Signal_center - (Signal_Bandwidth/2)
    Fin_High = Signal_center + (Signal_Bandwidth/2)
    
    #Calculate Nyquist rate
    Nyquist = float(Fs/2)
    
    #Calculate K variable
    if Fin_High <= Nyquist:
        K = 0
        
    elif Fin_High <= Fs + Nyquist:
        K = 1
    
    elif Fin_High <= 2*Fs + Nyquist:
        K = 2
        
    elif Fin_High <= 3*Fs + Nyquist:
        K = 3
        
    else:
        K = 4        
        
    #Calculate input alias
    Low_Input = (Fin_low - Fs*K)
    High_Input = (Fin_High - Fs*K)
    
    #Mirror input if value is negative
    if Low_Input < 0:
        Low_Input=Low_Input*-1
        High_Input=High_Input*-1    
        
        
    if  Fin_High < Nyquist:
        Low_Input=Fs-Fin_low
        High_Input=Fs-Fin_High

    #Return signal and its alias
    return [Fin_low, Fin_High,Low_Input, High_Input]
 

In [7]:
#This function will calculate the spurs due to the signal and their images
def DAC_Spur_Calculator(Fs, Signal_center, Signal_Bandwidth):

    #Call DAC inut function
    Input_info = DAC_Input_signal_calculator(Fs,Signal_center,Signal_Bandwidth)
    Nyquist = Fs/2
    #Set low and high boundaries to values from DAC function call
    fin_low = Input_info[0]
    fin_high = Input_info[1]
    fin_high_alias = Input_info[2]
    fin_low_alias = Input_info[3]
    
    #Create list for storing spur values
    HDlist=[]
        
    #For loop for calculating spurs HD2 upto HD5
    for n in range (2,6,1):
       
        if fin_high*n <= Nyquist:
            K = 0
        
        elif fin_high*n <= Fs + Nyquist:
            K = 1
    
        elif fin_high*n <= 2*Fs + Nyquist:
            K = 2
        
        elif fin_high*n <= 3*Fs + Nyquist:
            K = 3
        
        elif fin_high*n <= 4*Fs + Nyquist:
            K = 4
            
        elif fin_high*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6
        
        #Calculate spur locations
        HDn_high = (fin_high *n) - (Fs*K)
        HDn_low = (fin_low *n) - (Fs*K)
        
        #mirror signal
        if HDn_high < 0:
            HDn_high = HDn_high*-1
        
        
        if HDn_low <= 0:
            HDn_low = HDn_low*-1
    
        if HDn_high < 0:        #IF HDn is negative, mirror result 

                    Mirror_HDn_low = HDn_low * -1;
                    Mirror_HDn_high = HDn_high * -1;
                 
           #  print("mirrorRange of HD",n, "is", Mirror_HDn_Higher, "to", Mirror_HDn_Lower); # print range
                    temp_list = (Mirror_HDn_low,Mirror_HDn_high) 
                    HDlist.append(temp_list)        #Update list with calculated values                 
        
        elif  Nyquist < HDn_high:
            HDn_high = Nyquist
            temp_list = (HDn_low,HDn_high)
        else:
            # print("Range of HD",n, " is", HDn_Lower, "to", HDn_Higher); #Print range
                    temp_list = (HDn_high,HDn_low)
                    HDlist.append(temp_list)        #Update list with calculated values

                    
     #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_High = [HDlist[0][0],HDlist[1][0],HDlist[2][0],HDlist[3][0]]
  
    HD_High_list=[]
    
    for i in HD_High:
        if i <= 0 :
            i = 0 
            temp_list = (i)
                    #Update list with calculated values
            HD_High_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
                    #Update list with calculated values
            HD_High_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_low = [HDlist[0][1],HDlist[1][1],HDlist[2][1],HDlist[3][1]]
    
    
    HD_low_list=[]
    
    for i in HD_low:
        if i > Nyquist:
            i = Nyquist
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_list.append(temp_list)

    #Create spur values with relative list values
    HD2_value = [HD_High_list[0], HD_low_list[0]]
    HD3_value = [HD_High_list[1], HD_low_list[1]]
    HD4_value = [HD_High_list[2], HD_low_list[2]]
    HD5_value = [HD_High_list[3], HD_low_list[3]] 
                   

    #Set spur locations
    #HD2_value = [HDlist[0][0], HDlist[0][1]]
   # HD3_value = [HDlist[1][0], HDlist[1][1]]
   # HD4_value = [HDlist[2][0], HDlist[2][1]]
   # HD5_value = [HDlist[3][0], HDlist[3][1]]
    
    #Set spur image locations: Fs-spur locations = spur image location
    Ny_HD2 = [Fs-HDlist[0][0], Fs-HDlist[0][1]]
    Ny_HD3 = [Fs-HDlist[1][0], Fs-HDlist[1][1]]
    Ny_HD4 = [Fs-HDlist[2][0], Fs-HDlist[2][1]]
    Ny_HD5 = [Fs-HDlist[3][0], Fs-HDlist[3][1]]
    
     #Create array to check if list values are greater than Nyquist
    #This is used to set spurs greater than Nyquist = Nyquist, for plotting purposes
    
    HD_NY_High = [Fs-HDlist[0][0],Fs-HDlist[1][0],Fs-HDlist[2][0],Fs-HDlist[3][0]]
    
    HD_High_NY_list=[]
    
    for i in HD_NY_High:
        if i > Fs :
            i = Fs
            temp_list = (i)
                    #Update list with calculated values
            HD_High_NY_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
                    #Update list with calculated values
            HD_High_NY_list.append(temp_list)
    
    #Create array to check if list values are less than 0
    #This is used to set spurs less than 0 = 0, for plotting purposes
    
    HD_NY_low = [Fs-HDlist[0][1],Fs-HDlist[1][1],Fs-HDlist[2][1],Fs-HDlist[3][1]]
    
    HD_low_NY_list=[]
    
    for i in HD_NY_low:
        if i < Nyquist:
            i = Nyquist
            temp_list = (i)
            #Update list with calculated values
            HD_low_NY_list.append(temp_list)    
        else:
            i = i
            temp_list = (i)
            #Update list with calculated values
            HD_low_NY_list.append(temp_list)

    
    #Create spur values with relative list values
    HD2_Ny_value = [HD_High_NY_list[0], HD_low_NY_list[0]]
    HD3_Ny_value = [HD_High_NY_list[1], HD_low_NY_list[1]]
    HD4_Ny_value = [HD_High_NY_list[2], HD_low_NY_list[2]]
    HD5_Ny_value = [HD_High_NY_list[3], HD_low_NY_list[3]] 

    
    if (fin_low <= HD2_value[0])  and  (fin_high >= HD2_value[1]):
        print("HD2 overlaps")
    if (fin_low <= HD3_value[0])  and  (fin_high >= HD3_value[1]):
        print("HD3 overlaps")
    if (fin_low <= HD4_value[0])  and  (fin_high >= HD4_value[1]):
        print("HD4 overlaps")
    if (fin_low <= HD5_value[0])  and  (fin_high >= HD5_value[1]):
        print("HD5 overlaps")
    
    if (fin_low <= Ny_HD2[0])  and  (fin_high >= Ny_HD2[1]):
        print("HD2 overlaps")
    if (fin_low <= Ny_HD3[0])  and  (fin_high >= Ny_HD3[1]):
        print("HD3 overlaps")
    if (fin_low <= Ny_HD4[0])  and  (fin_high >= Ny_HD4[1]):
        print("HD4 overlaps")
    if (fin_low<= Ny_HD5[0])  and  (fin_high >= Ny_HD5[1]):
        print("HD5 overlaps")
        
        
    if (fin_low_alias <= HD2_value[0])  and  (fin_high_alias >= HD2_value[1]):
        print("HD2 overlaps")
    if (fin_low_alias  <= HD3_value[0])  and  (fin_high_alias >= HD3_value[1]):
        print("HD3 overlaps")
    if (fin_low_alias  <= HD4_value[0])  and  (fin_high_alias >= HD4_value[1]):
        print("HD4 overlaps")
    if (fin_low_alias  <= HD5_value[0])  and  (fin_high_alias >= HD5_value[1]):
        print("HD5 overlaps")
    #Return spurs and their images
    #return [HD2_value,HD3_value,HD4_value,HD5_value,Ny_HD2,Ny_HD3,Ny_HD4,Ny_HD5]
    return [HD2_value,HD3_value,HD4_value,HD5_value,HD2_Ny_value,HD3_Ny_value,HD4_Ny_value,HD5_Ny_value]

In [8]:
def Plot_RFDAC(Fs,Signal_center,Signal_Bandwidth):
    
    Input_plot = DAC_Input_signal_calculator(Fs, Signal_center, Signal_Bandwidth)
    Spur_plot = DAC_Spur_Calculator(Fs,Signal_center,Signal_Bandwidth)
    Nyquist = Fs/2
    
    #y-coordinates
    n1 = [5]
    n2 = [2,2]
    n3 = [3,3]
    n4 = [4,4]
    n5 = [5,5]
    nn = [5,0]
    
    #Set x and y data used for plotting
    spur_data = [Spur_plot[0],Spur_plot[1],Spur_plot[2],Spur_plot[3],Spur_plot[4],Spur_plot[5],Spur_plot[6],Spur_plot[7]]
    input_data = [Input_plot[2],Input_plot[3]]
    input_alias = [Input_plot[0],Input_plot[1]]
    y_data = [n2, n3, n4, n5,n2,n3,n4,n5]

    
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Harmonic numbers', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=spur_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink','purple','yellow','cyan','pink','purple','yellow'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    Fin_horz_plot = Lines(x=input_data,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot = Lines(x=[input_data[0],input_data[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot = Lines(x=[input_data[1],input_data[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_horz_plot_alias = Lines(x=input_alias,y=n5,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert1_plot_alias = Lines(x=[input_alias[0],input_alias[0]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Fin_vert2_plot_alias = Lines(x=[input_alias[1],input_alias[1]],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
    Nyquist_plot = Lines(x=[Nyquist,Nyquist],y=nn,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'harmonic number'])
    
    tt_fim = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency image'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Signal Frequency'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    spurs.marker='square'
    Fin_horz_plot.marker='square'
    Fin_horz_plot_alias.marker='square'
    Nyquist_plot.marker='square'
    #Plot
    fig = Figure(title='RFDAC frequency plan',axes=[ax_x, ax_y], marks=[Fin_horz_plot_alias,Fin_vert1_plot_alias,Fin_vert2_plot_alias,Fin_horz_plot,Fin_vert1_plot, Fin_vert2_plot,spurs,Nyquist_plot])
    spurs.tooltip=tt
    Fin_horz_plot.tooltip=tt_fim
    Fin_horz_plot_alias.tooltip=tt3
    Nyquist_plot.tooltip=Nyquisttt
    tools=Toolbar(figure=fig)

    display(fig,tools)

In [9]:
#Create sliders for selecting input information

Fs_slider = widgets.FloatSlider(value=3932.16,min=0,max=4000,step=10,description='Fs:')
Signal_Center_slider= widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal Center:')
Signal_Bandwidth = widgets.FloatSlider(value=100,min=50,max=200,step=50,description='Bandwidth:')

Fs_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Signal center:',continuous_update = False)
Signal_bandwidth_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Bandwidth:',continuous_update = False)
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((Signal_Bandwidth, 'value'), (Signal_bandwidth_text, 'value'))

left_box = VBox([Fs_slider, Signal_Center_slider, Signal_Bandwidth])
right_box = VBox([Fs_text, Signal_Center_text, Signal_bandwidth_text])
sel=HBox([left_box, right_box])

display(sel)

Fs= Fs_slider
Signal_center = Signal_Center_slider


#widgets.interact(Plot_RFADC,Fs=Fs_slider,Signal_center=Signal_Center_slider,Signal_Bandwidth=Signal_Bandwidth)

out = widgets.interactive_output(Plot_RFDAC, {'Fs': Fs, 'Signal_center': Signal_center, 'Signal_Bandwidth': Signal_Bandwidth})
display(out)

Output()

#  3) DUC Spectrum

The following spectrum will display the relevevant information for the Digital up conversion process.
If mix mode is on Nyquist zone 2 will show the relevant spurs.
If normal mode is on Nyquist zone 1 will show the relevant spurs. 
The spectrum shows both Nyquist zones so user does not need to make changes depending on which mode they operate.

In [10]:
def DUC_spur_calculator(Fs, Frequency_in, NCO):
    
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in
    F_image_calc = Fundamental - Fs

    if  Nyquist < F_image_calc:
        F_image = F_image_calc-Fs
        F_image = F_image *-1

    Nyquist = Fs/2 #Calculate Nyquist value
    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):
       
        
        if Fundamental*n < Nyquist:
            K = 0
        
        elif Fundamental*n <= Fs + Nyquist:
            K = 1
    
        elif Fundamental*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Fundamental*n <= 3*Fs + Nyquist:
            K = 3
        
        elif Fundamental*n <= 4*Fs + Nyquist:
            K = 4
            
        elif Fundamental*n <= 5*Fs + Nyquist:
            K = 5
        else:
            K = 6
        
        HDn = (Fundamental *n) - (Fs*K)
        
        if HDn < 0:
            HDn = HDn*-1


        if HDn <= 0:        #IF HDn is negative, mirror result 

                    Mirror_HDn = HDn * -1;
                 
                    temp_list = (Mirror_HDn) 
                    HDlist.append(temp_list)        #Update list with calculated values                 
        else:
                    temp_list = (HDn)
                    HDlist.append(temp_list)        #Update list with calculated values

    
    HD2_value = HDlist[0]
    HD3_value = HDlist[1]
    HD2_image = Fs - HD2_value
    HD3_image = Fs - HD3_value
    
    return [HD2_value, HD3_value,HD2_image,HD3_image]


In [11]:
def Plot_DUC(Fs,Frequency_in,NCO,HD2_noise,HD3_noise):
   
    Nyquist = float(Fs / 2)      #Calculate Nyquist rate
    Fundamental = NCO + Frequency_in
    F_image = Fundamental - Fs

    if  Nyquist <= F_image:
        F_image = F_image-Fs
        F_image = F_image *-1
    
    elif F_image < 0:
        Fimage= F_image*-1
        

        
    HD = DUC_spur_calculator(Fs,Frequency_in,NCO)

    HD2 = HD[0]
    HD3 = HD[1]
    HD2_image = HD[2]
    HD3_image = HD[3]
    
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD3 = [HD3, HD3]
    x_plot_HD2_image = [HD2_image, HD2_image]
    x_plot_HD3_image = [HD3_image, HD3_image]

    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]
    
    x_data = [x_plot_HD2,x_plot_HD3,x_plot_HD2_image,x_plot_HD3_image]
    y_data = [y_plot_HD2,y_plot_HD3,y_plot_HD2,y_plot_HD3]
    fund_x = [Fundamental,Fundamental]
    fim_x = [F_image, F_image]
    Fs_x = [Fs,Fs]
    fund_y = [0,-100]
    Nyquist1 = [Nyquist,Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spurs = Lines(x=x_data,y=y_data,scales={'x': x_sc, 'y': y_sc}, colors=['cyan','pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5},labels=['HD2','HD3'])
    Fundamental_plot = Lines(x=fund_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'],labels=['Fundamental'])
    Fim_plot = Lines(x=fim_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'],labels=['F image'])
    Nyquist_plot = Lines(x=Nyquist1,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'],labels=['Nyquist'])
    Fs_plot = Lines(x=Fs_x,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'],labels=['Fs'])
   
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency', 'dB level'])

    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Frequency'])
    Nyquisttt = Tooltip(fields=['x'], formats=[''], labels=['Nyquist Frequency'])
    
    spurs.marker='square'
    Fundamental_plot.marker='square'
    Fim_plot.marker='square'
    Fs_plot.marker='square'
    Nyquist_plot.marker = 'square'

    
    fig = Figure(title='DUC Spectrum',axes=[ax_x, ax_y], marks=[spurs,Fundamental_plot,Fs_plot,Nyquist_plot,Fim_plot],
                     display_legend=True)

    labels = ['spurs','Fundamental_plot','Fs_plot','Nyquist_plot','Fim_plot']
    
    spurs.tooltip=tt
    Fundamental_plot.tooltip=tt
    Nyquist_plot.tooltip=Nyquisttt
    Fs_plot.tooltip=tt
    Fim_plot.tooltip=tt
    tools=Toolbar(figure=fig)
    
        
    display(fig, tools)

In [33]:
Fs_slider = widgets.FloatSlider(value=3932.16,min=1900,max=4000,step=100,description='Fs:')
Signal_Center_slider = widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal center:')
NCO_slider = widgets.FloatSlider(value=3500,min=100,max=4000, step = 50,description='NCO:')
HD2_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10,description='HD2 noise:')
HD3_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10,description='HD3 noise:')
#widgets.interact(Plot_DUC,Fs=Fs_slider,Frequency_in=Signal_Center_slider,NCO=NCO_slider,HD2_noise=HD2_slider,HD3_noise=HD3_slider)



Fs_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Signal center:',continuous_update = False)
NCO_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='NCO:',continuous_update = False)
HD2_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='HD2 noise:',continuous_update = False)
HD3_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='HD3 noise:',continuous_update = False)
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((NCO_slider, 'value'), (NCO_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))

left_box = VBox([Fs_slider, Signal_Center_slider, NCO_slider,HD2_slider,HD3_slider])
right_box = VBox([Fs_text, Signal_Center_text, NCO_text,HD2_text,HD3_text])

sel=HBox([left_box, right_box])

display(sel)

Fs= Fs_slider
Signal_center = Signal_Center_slider
NCO = NCO_slider

#widgets.interact(Plot_RFADC,Fs=Fs_slider,Signal_center=Signal_Center_slider,Signal_Bandwidth=Signal_Bandwidth)

out = widgets.interactive_output(Plot_DUC, {'Fs': Fs, 'Frequency_in': Signal_center, 'NCO': NCO,
                                              'HD2_noise':HD2_slider,'HD3_noise':HD3_slider })
display(out)

Output()

# DDC Spectrum

The following spectrum displays the relevant information for the Digital down conversion process. Note the values displayed are the decimated values. Any values exceeding the decimated Nquist value are folded back until they are present in the display band.

In [13]:
def Equation_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    Nyquist = Fs/2 #Calculate Nyquist value
    
    # Calculate K variable, K increases as conditions are met
    
    if Frequency_in < Nyquist:
        K=0
        
    elif Frequency_in > Nyquist:
        K = 1
        
    elif Frequency_in > Fs + Nyquist:
        K = 2
    
    elif Frequency_in > 2*Fs + Nyquist:
        K = 3
        
    elif Frequency_in > 3*Fs + Nyquist:
        K = 4
        
    else:
        K = 5
        
    ## Potentially add error statement if input frequency reaches unrealistic value 
    
    #Calculate input alias before NCO shift
    Input = (Frequency_in - Fs*K)
    
    #Mirror input if value is negative
    if Input < 0:
        Input=Input*-1
     
    #Shift input by NCO
    Input_NCO = (Input) - NCO
    
    #Calculate final Answer for input alias 
    Answer = Input_NCO + (Interleaved_ADC * Fs/Decimation)

    if Answer < 0:
        Answer = Answer * -1
    
    #Use decimated Nyquist and Fs values for calculating signal in 1st decimated Nyquist zone
    Decimated_Nyquist = Nyquist/Decimation
    Decimated_Fs = Fs/Decimation
    
    
    #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone
    stopprocess = False

    while stopprocess == False:

        if Answer > Decimated_Nyquist:
            Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
            if Answer <= Decimated_Nyquist:
                stopprocess = True

    return Answer

In [14]:
def DDC_spur_calculator(Fs, Frequency_in, NCO, Decimation , Interleaved_ADC):
    
    
    Nyquist = Fs/2 #Calculate Nyquist value
    HDlist = [] #Create a list for storing loop values
    
    # Calculate K variable, K increases as conditions are met
    
    for n in range (2,4,1):
       
        
        if Frequency_in*n < Nyquist:
            K = 0
        
        elif Frequency_in*n <= Fs + Nyquist:
            K = 1
    
        elif Frequency_in*n <= 2*Fs + Nyquist:
            K = 2
        
        elif Frequency_in*n <= 3*Fs + Nyquist:
            K = 3
        
        else:
            K = 4

        hdn_orig = (Frequency_in*n) - (Fs*K)
        
        hdn_image = Fs - hdn_orig
        
        if hdn_orig < 0:
            hdn_orig = hdn_orig*-1
            
        if hdn_image < 0:
            hdn_image = hdn_image*-1
            
        hd_NCO = hdn_orig - NCO
        hd_image_nco = hdn_image - NCO
        #Calculate final Answer for input alias 
        Answer = hd_NCO + (Interleaved_ADC * Fs/Decimation)
        Answer2 = hd_image_nco + (Interleaved_ADC * Fs/Decimation)
        
        Decimated_Nyquist = Nyquist/Decimation
        Decimated_Fs = Fs/Decimation
    
        
        #Use following process to continuously subtract decimated FS until value is displayed in first decimated Nyquist zone
        stopprocess = False

        while stopprocess == False:
            if Answer > Decimated_Nyquist:
                Answer = Answer-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer <= Decimated_Nyquist:
                    stoprocess = True
                    temp_list = (Answer)
                    HDlist.append(temp_list) 
                    Answer = 0
                    break
                stopprocess = False

        stopprocess2 = False
        
        while stopprocess2 == False:
            if Answer2 > Decimated_Nyquist:
                Answer2 = Answer2-Decimated_Fs
            
            #Set to true when value is below decimated Nyquist
                if Answer2 <= Decimated_Nyquist:
                    stoprocess = True
                    temp_list = (Answer2)
                    HDlist.append(temp_list) 
                    Answer2 = 0
                    break

    
    HD2_value = HDlist[0]
    HD3_value = HDlist[2]
    HD2_image_value = HDlist[1]
    HD3_image_value = HDlist[3]
    
    return [HD2_value, HD3_value, HD2_image_value, HD3_image_value]


In [15]:
def Plot_DDC(Fs, Frequency_in,NCO,HD2_noise,HD3_noise, Decimation, Interleaved_ADC):
    
    Nyquist = Fs/2
    Decimated_Nyquist = Nyquist/Decimation
    
    input_alias = Equation_calculator(Fs,Frequency_in,NCO,Decimation,Interleaved_ADC)
    
    HD = DDC_spur_calculator(Fs,Frequency_in,NCO, Decimation, Interleaved_ADC)

    
    y_plot_HD2 = [-1*HD2_noise,-100]
    y_plot_HD3 = [-1*HD3_noise,-100]

    HD2 = HD[0]
    HD3 = HD[1]
    HD2_image = HD[2]
    HD3_image = HD[3]
    
    x_plot_HD2 = [HD2, HD2]
    x_plot_HD2_image = [HD2_image, HD2_image]
    x_plot_HD3 = [HD3, HD3]
    x_plot_HD3_image = [HD3_image, HD3_image]
    
    x_data_HD2 = [x_plot_HD2]
    x_data_HD3 = [x_plot_HD3]
    x_data_HD2_image = [x_plot_HD2_image]
    x_data_HD3_image = [x_plot_HD3_image]
    y_data_HD2 = [y_plot_HD2]
    y_data_HD3 = [y_plot_HD3]
    f_in = [input_alias,input_alias]
    
    fund_y = [0,-100]
    Decimated_Nyquist_plot = [Decimated_Nyquist,Decimated_Nyquist]
    Negative_Nyquist = [-1*Decimated_Nyquist,-1*Decimated_Nyquist]
    
    x_sc = LinearScale()
    y_sc = LinearScale()
    ax_x = Axis(label='Frequency(MHz)', scale=x_sc, tick_format='0.0f')
    ax_y = Axis(label='Amplitude(dB)', scale=y_sc,
               orientation='vertical', tick_format='0.2f')
    
    #Create lines for plotting
    spur_HD2 = Lines(x=x_data_HD2,y=y_data_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    spur_HD3 = Lines(x=x_data_HD3,y=y_data_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    spur_HD2_image = Lines(x=x_data_HD2_image,y=y_data_HD2,scales={'x': x_sc, 'y': y_sc}, colors=['cyan'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    spur_HD3_image = Lines(x=x_data_HD3_image,y=y_data_HD3,scales={'x': x_sc, 'y': y_sc}, colors=['pink'],enable_move=False,
                   hovered_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
                   unhovered_style={'opacity': 0.5})
    f_in = Lines(x=f_in,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['blue'])
  
    Decimated_Nyquist_plot = Lines(x=Decimated_Nyquist_plot,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
    Negative_Nyquist_plot = Lines(x=Negative_Nyquist,y=fund_y,scales={'x': x_sc, 'y': y_sc},colors=['red'])
  
   
    tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Frequency in', 'dB level'])
    tt_HD2 = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD2', 'dB level'])
    tt_HD3 = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD3', 'dB level'])
    tt_HD2_image = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD2 image', 'dB level'])
    tt_HD3_image = Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['HD3 image', 'dB level'])
    tt3 = Tooltip(fields=['x'], formats=[''], labels=['Frequency'])
    
    spur_HD2.marker='square'
    spur_HD3.marker='square'
    spur_HD2_image.marker='square'
    spur_HD3_image.marker='square'
    f_in.marker='square'
    Decimated_Nyquist_plot.marker = 'square'
    Negative_Nyquist_plot.marker = 'square'

    
    fig = Figure(title='DDC Spectrum',axes=[ax_x, ax_y], marks=[spur_HD2,spur_HD3,spur_HD2_image,spur_HD3_image,f_in,Decimated_Nyquist_plot,Negative_Nyquist_plot])
    
    spur_HD2_image.tooltip=tt_HD2_image
    spur_HD3_image.tooltip=tt_HD3_image
    spur_HD2.tooltip=tt_HD2
    spur_HD3.tooltip=tt_HD3
    f_in.tooltip=tt
    Decimated_Nyquist_plot.tooltip=tt
    tools=Toolbar(figure=fig)
    
        
    display(fig, tools)

In [37]:
Fs_slider = widgets.FloatSlider(value=3932.16,min=1900,max=4000,step=100,description='Fs:')
Signal_Center_slider = widgets.FloatSlider(value=3500,min=100,max=4000,step=50,description='Signal center:')
NCO_slider = widgets.FloatSlider(value=3500,min=100,max=4000, step = 50,description='NCO:')
HD2_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10,description='HD2 noise:')
HD3_slider = widgets.FloatSlider(value=10, min=0, max=200, step = 10,description='HD3 noise:')
#widgets.interact(Plot_DUC,Fs=Fs_slider,Frequency_in=Signal_Center_slider,NCO=NCO_slider,HD2_noise=HD2_slider,HD3_noise=HD3_slider)

#Create button widgets to control variables
Decimation_buttons = widgets.RadioButtons(
    options=[2,4,8],
    value=8,
    description='Decimation:',
    disabled=False
)

Interleaved_ADC_buttons = widgets.RadioButtons(
    options=[4,8],
    value=8,
    description='Int ADCs:',
    disabled=False
)


Fs_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Fs:',continuous_update = False)
Signal_Center_text = widgets.BoundedFloatText(value=3932.16,min=0,max=4000,step=0.1,description='Signal center:',continuous_update = False)
NCO_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='NCO:',continuous_update = False)
HD2_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='HD2 noise:',continuous_update = False)
HD3_text = widgets.BoundedFloatText(value=1000,min=0,max=2000,step=0.1,description='HD3 noise:',continuous_update = False)
l = widgets.link((Fs_slider, 'value'), (Fs_text, 'value'))
l = widgets.link((Signal_Center_slider, 'value'), (Signal_Center_text , 'value'))
l = widgets.link((NCO_slider, 'value'), (NCO_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))
l = widgets.link((HD2_slider, 'value'), (HD2_text, 'value'))

left_box = VBox([Fs_slider, Signal_Center_slider, NCO_slider,HD2_slider,HD3_slider])
right_box = VBox([Fs_text, Signal_Center_text, NCO_text,HD2_text,HD3_text])
button_box = VBox([Interleaved_ADC_buttons,Decimation_buttons])

sel=HBox([left_box, right_box,button_box])

display(sel)

Fs= Fs_slider
Signal_center = Signal_Center_slider
NCO = NCO_slider

#widgets.interact(Plot_RFADC,Fs=Fs_slider,Signal_center=Signal_Center_slider,Signal_Bandwidth=Signal_Bandwidth)

out = widgets.interactive_output(Plot_DDC, {'Fs': Fs, 'Frequency_in': Signal_center, 'NCO': NCO,
                                              'HD2_noise':HD2_slider,'HD3_noise':HD3_slider,'Decimation':Decimation_buttons,
                                            'Interleaved_ADC':Interleaved_ADC_buttons})
display(out)



Output()